In [1]:
import SimpleITK as sitk
import os
import glob
import json
import re

In [2]:
input_path = "C:/Users/lydia/OneDrive/Work/DigitalAssets/_UPENN_fMOST/BIL_DATA/proj/u19zeng/fMOST_downsampled/brain"
output_path = "C:/Users/lydia/OneDrive/Work/DigitalAssets/_UPENN_fMOST/BIL_DATA/proj/u19zeng/fMOST_downsampled/fixed_headers"

In [3]:
# collate input files
samples = {}
template = '*.json'

for f in glob.glob( os.path.join( input_path, template )) :

    # read json file
    with open(f,) as jf :
        data = json.load(jf)
        
    sample_id = data['SampleInfo']['SampleID']
    print('- ', sample_id)
        
    samples[sample_id] = data
    samples[sample_id]['channels'] = {}
    
    # parse filenames for channel and downsampling info
    for c in ['red','green','blue'] :
    
        t2 = 'mouse%s%s*.tif' % (sample_id,c) 
    
        for f2 in glob.glob( os.path.join( input_path, t2 )) :
        
            print('-- ' + f2)
            
            samples[sample_id]['channels'][c] = {}
            samples[sample_id]['channels'][c]['input_path'] = f2
            
            bname = os.path.basename(f2)
            bname = os.path.splitext(bname)[0]
            
            samples[sample_id]['channels'][c]['output_path'] = os.path.join(output_path, bname + '.nii.gz')
            
            nums = re.findall('[0-9]+', bname)
            
            samples[sample_id]['channels'] [c]['downsample'] = [int(nums[1]),int(nums[1]),int(nums[2])]
            samples[sample_id]['channels'] [c]['spacing'] = [1,1,1]
            
            for d in range(3) :
                samples[sample_id]['channels'][c]['spacing'][d] = \
                    samples[sample_id]['channels'][c] ['downsample'][d] * \
                    samples[sample_id]['SampleInfo']['VoxelResolution '][d] * \
                    0.001 # in mm    

-  182709
-- C:/Users/lydia/OneDrive/Work/DigitalAssets/_UPENN_fMOST/BIL_DATA/proj/u19zeng/fMOST_downsampled/brain\mouse182709green_xy32z8.tif
-  182724
-- C:/Users/lydia/OneDrive/Work/DigitalAssets/_UPENN_fMOST/BIL_DATA/proj/u19zeng/fMOST_downsampled/brain\mouse182724red_xy32z8.tif
-- C:/Users/lydia/OneDrive/Work/DigitalAssets/_UPENN_fMOST/BIL_DATA/proj/u19zeng/fMOST_downsampled/brain\mouse182724green_xy32z8.tif
-  191807
-- C:/Users/lydia/OneDrive/Work/DigitalAssets/_UPENN_fMOST/BIL_DATA/proj/u19zeng/fMOST_downsampled/brain\mouse191807red_xy32z8.tif
-- C:/Users/lydia/OneDrive/Work/DigitalAssets/_UPENN_fMOST/BIL_DATA/proj/u19zeng/fMOST_downsampled/brain\mouse191807green_xy32z8.tif
-  191812
-- C:/Users/lydia/OneDrive/Work/DigitalAssets/_UPENN_fMOST/BIL_DATA/proj/u19zeng/fMOST_downsampled/brain\mouse191812red_xy32z8.tif
-- C:/Users/lydia/OneDrive/Work/DigitalAssets/_UPENN_fMOST/BIL_DATA/proj/u19zeng/fMOST_downsampled/brain\mouse191812green_xy32z8.tif
-  192343
-- C:/Users/lydia/OneDriv

In [4]:
# process samples

for s in samples :
    
    print("converting %s" % s)
    
    for c in samples[s]['channels'] :
    
        info = samples[s]['channels'][c]
        vol = sitk.ReadImage(info['input_path'])
        vol.SetSpacing( info['spacing'])
    
        #
        # save file to output directory
        #
        sitk.WriteImage( vol, info['output_path'], True )


converting 182709
converting 182724
converting 191807
converting 191812
converting 192343
converting 194069
